# Robin Hood Trading Demo

Presented here is a demonstration of how to use the robin.py module from the rh_trading repository on my github (https://github.com/michaelray1). The code is still very buggy, as you will see in the demonstration, so I need to work out a lot of those still.

In [7]:
#Import the robin.py module and numpy
import robin as rb
import numpy as np

In [4]:
#Create a statistics object, then use that to create a metrics object
stats = rb.statistics()
metrics = rb.metrics(statistics = stats)

#Now use these two to create a neural network object. You also need a username and password for this. Enter them below.
un = ''
pw = ''
nn = rb.nn(metrics = metrics, statistics = stats, un = un, pw = pw)

So now we've got all the necessary ingredients for the setup of our neural network. The last thing we need is some data to train the neural network. All we need to provide is the stock tickers for whatever stocks we are interested in analyzing. I've got a file saved here that has about 25 blue chip stocks (expensive, safe stocks) to use for this demonstration. Let's load in the tickers.

In [8]:
#Load in blue chip stock tickers and print out what it looks like
blue_chips = np.load('blue_chip_stock_tickers.npz')['arr_0']
print(blue_chips)

['IBM' 'XOM' 'CVX' 'PG' 'MMM' 'JNJ' 'MCD' 'WMT' 'KO' 'BA' 'CAT' 'JPM'
 'HPQ' 'VZ' 'T' 'DD' 'MRK' 'DIS' 'HD' 'MSFT' 'AXP' 'BAC' 'PFE' 'GE' 'INTC'
 'AA' 'C' 'GM']


In [9]:
#Use neural network functions to get training and testing data
nn.get_tt_data(inputSymbols = blue_chips)

completed collecting data for IBM (1 of 28 stocks done)
completed collecting data for XOM (2 of 28 stocks done)
completed collecting data for CVX (3 of 28 stocks done)
completed collecting data for PG (4 of 28 stocks done)
completed collecting data for MMM (5 of 28 stocks done)
completed collecting data for JNJ (6 of 28 stocks done)
completed collecting data for MCD (7 of 28 stocks done)
completed collecting data for WMT (8 of 28 stocks done)
completed collecting data for KO (9 of 28 stocks done)
completed collecting data for BA (10 of 28 stocks done)
completed collecting data for CAT (11 of 28 stocks done)
completed collecting data for JPM (12 of 28 stocks done)
completed collecting data for HPQ (13 of 28 stocks done)
completed collecting data for VZ (14 of 28 stocks done)
completed collecting data for T (15 of 28 stocks done)
completed collecting data for DD (16 of 28 stocks done)
completed collecting data for MRK (17 of 28 stocks done)
completed collecting data for DIS (18 of 28 sto

The testing and training data is crucial to creating a successful neural network. In a sense, this is the main function that implements my trading strategy. Before I give a quick explanation of what the training/testing data is, let's look at it.

In [14]:
nn.testing

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [13]:
nn.training

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 1., 0.],
       [0., 1., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

Each group of 4 ones and zeros here represents one stock's data (if you count up all the rows shown, you get 28 which represents all the blue chip stocks we used). The rightmost value in each row is the "expected output". By default (you can change all these settings in the keyword arguments of nn.get_tt_data), this tells us whether the stock went up by 4% within 10 days of the end of the data that the neural network has access to. Essentially, we truncate the data that the neural network has access to. Then we use that shortened data set to try to predict the "future" (we have the "future" data already, we are using it to see if the neural network can make accurate predictions).

Now, the other three numbers represent the different metrics. At current, there's only three metrics coded in, but I plan to expand this later. The three metrics coded in for now are bbands_bottom, ma_crossover, and rsi_crossover. bbands_bottom gives 1 if the stock's price is below one standard deviation from its mean, and 0 if it isn't that low. ma_crossover gives 1 if the short period (25 days) moving average is higher than the long period moving average (250 days). rsi_crossover is very similar to ma_crossover, but it uses the so called "relative strength index" as a measure of how the stock is doing.

Once the neural network has the metrics, the idea is that it will weight each metric by a given amount (call these weights w_i). Then, it takes a look at the "total prediction" which is the sum of w_i * m_i where m_i is the i'th metric. Basically, we just look at a weighted sum of the metrics. Then a reLU activation function is applied where if the total prediction is higher than some specified value, then the neural network predicts a "1" which corresponds to "buy the stock". Anything below the specified value of the total prediction means that the neural network returns a "0" which corresponds to "don't buy the stock". 

Now, what the neural network is doing, is it's using the last column of our training data (the "expected output") to optimize the weights, w_i. By using the training/testing data to optimize the weights, the idea is that we can let the computer figure out which metrics are actually the best indicators of whether a stock will go up or down. Perhaps it's actually some funky combination of all the metrics that best indicates whether a stock will go up or down. Now let's see how to check the accuracy of our neural network and also use it to predict things about other stocks. This is where the code gets buggy and needs some work.

In [15]:
#Build the neural network with default settings for size of layers
nn.build_network()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
#Train the network
nn.train_network()

Train on 22 samples, validate on 6 samples
Epoch 1/50
22/22 [==============================] - 0s 9ms/sample - loss: 0.0490 - val_loss: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 0s 300us/sample - loss: 0.0458 - val_loss: 0.0000e+00
Epoch 3/50
22/22 [==============================] - 0s 253us/sample - loss: 0.0458 - val_loss: 0.0000e+00
Epoch 4/50
22/22 [==============================] - 0s 262us/sample - loss: 0.0459 - val_loss: 0.0000e+00
Epoch 5/50
22/22 [==============================] - 0s 304us/sample - loss: 0.0458 - val_loss: 0.0000e+00
Epoch 6/50
22/22 [==============================] - 0s 254us/sample - loss: 0.0456 - val_loss: 0.0000e+00
Epoch 7/50
22/22 [==============================] - 0s 327us/sample - loss: 0.0455 - val_loss: 0.0000e+00
Epoch 8/50
22/22 [==============================] - 0s 319us/sample - loss: 0.0455 - val_loss: 0.0000e+00
Epoch 9/50
22/22 [==============================] - 0s 326us/sample - loss: 0.0456 - val_loss: 0.0000e+00
Epoch

In [18]:
#Test accuracy should tell us exactly how many the neural network correctly predicted from the testing/training set.
#Be careful though, the network is trained on the same data set that we are using "test_accuracy" on.
#So, if there is underlying bias in the data set, or if we overtrain on our data set, then test_accuracy will
#tell us that our neural network has done very well, when it may not predict other stocks well at all.
nn.test_accuracy()

400 Client Error: Bad Request for url: https://api.robinhood.com/quotes/historicals/?symbols=&interval=day&span=year&bounds=regular


ZeroDivisionError: division by zero

In [24]:
#We can also of course use the trained network to predict other stocks (this is what we ultimately want to do)
#Not sure why this error is coming up. It seems to me that input_data is, in fact, of size (3,)
nn.predict(stock='AAPL')

ValueError: Error when checking input: expected input_1 to have shape (3,) but got array with shape (1,)